In [1]:
import numpy as np
import deepchem as dc
import numpy as np
from deepchem.feat import GraphData
import pandas as pd
from deepchem.feat import DMPNNFeaturizer
from datetime import datetime
from tqdm import tqdm
from sklearn.metrics import roc_auc_score


No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\models\torch_models\__init__.py)


In [2]:
def get_graphs(weighted_adj_matrix):
    node_features = weighted_adj_matrix.diagonal().reshape(-1, 1)
    edges = np.array(np.nonzero(weighted_adj_matrix))
    edge_weights = weighted_adj_matrix[edges.T[:, 0], edges.T[:, 1]]
    graph = GraphData(node_features=node_features, edge_index=edges, edge_features=edge_weights.reshape(-1, 1))
    return graph

In [3]:
TASKS = [
'alcoholic', 'aldehydic', 'alliaceous', 'almond', 'amber', 'animal',
'anisic', 'apple', 'apricot', 'aromatic', 'balsamic', 'banana', 'beefy',
'bergamot', 'berry', 'bitter', 'black currant', 'brandy', 'burnt',
'buttery', 'cabbage', 'camphoreous', 'caramellic', 'cedar', 'celery',
'chamomile', 'cheesy', 'cherry', 'chocolate', 'cinnamon', 'citrus', 'clean',
'clove', 'cocoa', 'coconut', 'coffee', 'cognac', 'cooked', 'cooling',
'cortex', 'coumarinic', 'creamy', 'cucumber', 'dairy', 'dry', 'earthy',
'ethereal', 'fatty', 'fermented', 'fishy', 'floral', 'fresh', 'fruit skin',
'fruity', 'garlic', 'gassy', 'geranium', 'grape', 'grapefruit', 'grassy',
'green', 'hawthorn', 'hay', 'hazelnut', 'herbal', 'honey', 'hyacinth',
'jasmin', 'juicy', 'ketonic', 'lactonic', 'lavender', 'leafy', 'leathery',
'lemon', 'lily', 'malty', 'meaty', 'medicinal', 'melon', 'metallic',
'milky', 'mint', 'muguet', 'mushroom', 'musk', 'musty', 'natural', 'nutty',
'odorless', 'oily', 'onion', 'orange', 'orangeflower', 'orris', 'ozone',
'peach', 'pear', 'phenolic', 'pine', 'pineapple', 'plum', 'popcorn',
'potato', 'powdery', 'pungent', 'radish', 'raspberry', 'ripe', 'roasted',
'rose', 'rummy', 'sandalwood', 'savory', 'sharp', 'smoky', 'soapy',
'solvent', 'sour', 'spicy', 'strawberry', 'sulfurous', 'sweaty', 'sweet',
'tea', 'terpenic', 'tobacco', 'tomato', 'tropical', 'vanilla', 'vegetable',
'vetiver', 'violet', 'warm', 'waxy', 'weedy', 'winey', 'woody'
]
print("No of tasks: ", len(TASKS))

No of tasks:  138


In [4]:
import deepchem as dc
from openpom.feat.graph_featurizer import GraphFeaturizer, GraphConvConstants
from openpom.utils.data_utils import get_class_imbalance_ratio

In [6]:
from deepchem.feat import DMPNNFeaturizer
import deepchem as dc


featurizer = DMPNNFeaturizer()

smiles_field = 'nonStereoSMILES'


loader = dc.data.CSVLoader(
    tasks=TASKS,  
    feature_field=smiles_field,
    featurizer=featurizer
)



In [7]:

dataset = loader.create_dataset(inputs=["DMPNNpruned_without hydrogen_curated_GS_LF_merged_4812_QM_cleaned.csv"])
n_tasks = len(dataset.tasks)
dataset.X
len(dataset.X)


4812

In [8]:
data = np.load("DMPNNpruned_graph_data_cleaned.npz",allow_pickle=True)
mtx_list = data['mtx']
qm_graphs = []
for mtx in mtx_list:
    qm_graphs.append(get_graphs(mtx))
QM_X = np.asarray(qm_graphs)
QM_X
len(QM_X)


4812

In [9]:
X = np.asarray([i for i in zip(dataset.X, QM_X)])
new_dataset = dc.data.NumpyDataset(X=X, y=dataset.y, n_tasks=138, ids=dataset.ids)

In [10]:
new_dataset

<NumpyDataset X.shape: (4812, 2), y.shape: (4812, 138), w.shape: (4812, 1), task_names: [  0   1   2 ... 135 136 137]>

In [14]:
new_dataset

<NumpyDataset X.shape: (4812, 2), y.shape: (4812, 138), w.shape: (4812, 1), task_names: [  0   1   2 ... 135 136 137]>

In [ ]:


randomstratifiedsplitter = dc.splits.RandomStratifiedSplitter()
train_dataset, test_dataset, valid_dataset = randomstratifiedsplitter.train_valid_test_split(new_dataset, frac_train = 0.8, frac_valid = 0.1, frac_test = 0.1, seed = 1)

In [16]:
print(len(train_dataset))
print(len(valid_dataset))
print(len(test_dataset))

3839
485
488


In [17]:
dc.data.DiskDataset.from_numpy(X=train_dataset.X,
                    y= train_dataset.y,
                    w= train_dataset.w,
                    ids=train_dataset.ids,
                    tasks=TASKS,
                    data_dir="./train_split")

<DiskDataset X.shape: (3839, 2), y.shape: (3839, 138), w.shape: (3839, 1), task_names: ['alcoholic' 'aldehydic' 'alliaceous' ... 'weedy' 'winey' 'woody']>

In [18]:
dc.data.DiskDataset.from_numpy(X=valid_dataset.X,
                    y= valid_dataset.y,
                    w= valid_dataset.w,
                    ids=valid_dataset.ids,
                    tasks=TASKS,
                    data_dir="./valid_split")

<DiskDataset X.shape: (485, 2), y.shape: (485, 138), w.shape: (485, 1), ids: ['CCC(=O)C(=O)O' 'O=C(O)CCCCC(=O)O' 'O=C(O)c1ccccc1' ...
 'CC(C)C(NC(=O)CCC(N)C(=O)O)C(=O)NCC(=O)O' 'CCC1SC(CC(C)C)=NC1C'
 'O=c1[nH]cnc2c1ncn2C1OC(COP(=O)(O)O)C(O)C1O'], task_names: ['alcoholic' 'aldehydic' 'alliaceous' ... 'weedy' 'winey' 'woody']>

In [19]:
dc.data.DiskDataset.from_numpy(X=test_dataset.X,
                    y= test_dataset.y,
                    w= test_dataset.w,
                    ids=test_dataset.ids,
                    tasks=TASKS,
                    data_dir="./test_split")

<DiskDataset X.shape: (488, 2), y.shape: (488, 138), w.shape: (488, 1), ids: ['CC(O)CN' 'O=Cc1ccccc1' 'CC(=O)C(C)=O' ... 'CCCCCCCC(C)=NO'
 'CCCC=CC1OCC(O)CO1' 'CC(C)C1CCC2CC(=O)CC(C)C2(C)C1'], task_names: ['alcoholic' 'aldehydic' 'alliaceous' ... 'weedy' 'winey' 'woody']>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List, Tuple, Union, Optional, Callable, Dict

from deepchem.models.losses import Loss, L2Loss
from deepchem.models.torch_models.torch_model import TorchModel
from deepchem.models.optimizers import Optimizer, LearningRateSchedule

from openpom.layers.pom_ffn import CustomPositionwiseFeedForward
from openpom.utils.loss import CustomMultiLabelLoss
from openpom.utils.optimizer import get_optimizer

try:
    import dgl
    from dgl import DGLGraph
    from dgl.nn.pytorch import Set2Set
    from openpom.layers.pom_mpnn_gnn import CustomMPNNGNN
except (ImportError, ModuleNotFoundError):
    raise ImportError('This module requires dgl and dgllife')


class MPNNPOM(nn.Module):

    def __init__(self,
                 n_tasks: int,
                 node_out_feats: int = 64,
                 edge_hidden_feats: int = 128,
                 edge_out_feats: int = 64,
                 num_step_message_passing: int = 3,
                 mpnn_residual: bool = True,
                 message_aggregator_type: str = 'sum',
                 mode: str = 'classification',
                 number_atom_features: int = 133,
                 number_bond_features: int = 14,
                 n_classes: int = 1,
                 nfeat_name: str = 'x',
                 efeat_name: str = 'edge_attr',
                 readout_type: str = 'set2set',
                 num_step_set2set: int = 6,
                 num_layer_set2set: int = 3,
                 ffn_hidden_list: List = [300],
                 ffn_embeddings: int = 256,
                 ffn_activation: str = 'relu',
                 ffn_dropout_p: float = 0.0,
                 ffn_dropout_at_input_no_act: bool = True):

        if mode not in ['classification', 'regression']:
            raise ValueError(
                "mode must be either 'classification' or 'regression'")

        super(MPNNPOM, self).__init__()

        self.n_tasks: int = n_tasks
        self.mode: str = mode
        self.n_classes: int = n_classes
        self.nfeat_name: str = nfeat_name
        self.efeat_name: str = efeat_name
        self.readout_type: str = readout_type
        self.ffn_embeddings: int = ffn_embeddings
        self.ffn_activation: str = ffn_activation
        self.ffn_dropout_p: float = ffn_dropout_p

        if mode == 'classification':
            self.ffn_output: int = n_tasks * n_classes
        else:
            self.ffn_output = n_tasks

        self.mpnn: nn.Module = CustomMPNNGNN(
            node_in_feats=number_atom_features,
            node_out_feats=node_out_feats,
            edge_in_feats=number_bond_features,
            edge_hidden_feats=edge_hidden_feats,
            num_step_message_passing=num_step_message_passing,
            residual=mpnn_residual,
            message_aggregator_type=message_aggregator_type)
        
        QM_node_out_features = 50
        
        self.QM_mpnn: nn.Module = CustomMPNNGNN(
            node_in_feats=1,
            node_out_feats=QM_node_out_features,
            edge_in_feats=1,
            edge_hidden_feats=16,
            num_step_message_passing=2,
            residual=True,
            message_aggregator_type='sum')

        self.project_edge_feats: nn.Module = nn.Sequential(
            nn.Linear(number_bond_features, edge_out_feats), nn.ReLU())

        if self.readout_type == 'set2set':
            self.readout_set2set: nn.Module = Set2Set(
                input_dim=node_out_feats + edge_out_feats,
                n_iters=num_step_set2set,
                n_layers=num_layer_set2set)
            ffn_input: int = 2 * (node_out_feats + edge_out_feats)
        elif self.readout_type == 'global_sum_pooling':
            ffn_input = node_out_feats + edge_out_feats
        else:
            raise Exception("readout_type invalid")

        if ffn_embeddings is not None:
            d_hidden_list: List = ffn_hidden_list + [ffn_embeddings]

        self.ffn: nn.Module = CustomPositionwiseFeedForward(
            d_input=ffn_input + QM_node_out_features,
            
            d_hidden_list=d_hidden_list,
            d_output=self.ffn_output,
            activation=ffn_activation,
            dropout_p=ffn_dropout_p,
            dropout_at_input_no_act=ffn_dropout_at_input_no_act)

    def _readout(self, g: DGLGraph, node_encodings: torch.Tensor,
                 edge_feats: torch.Tensor) -> torch.Tensor:

        g.ndata['node_emb'] = node_encodings
        g.edata['edge_emb'] = self.project_edge_feats(edge_feats)

        def message_func(edges) -> Dict:
            """
            The message function to generate messages
            along the edges for DGLGraph.send_and_recv()
            """
            src_msg: torch.Tensor = torch.cat(
                (edges.src['node_emb'], edges.data['edge_emb']), dim=1)
            return {'src_msg': src_msg}

        def reduce_func(nodes) -> Dict:
            """
            The reduce function to aggregate the messages
            for DGLGraph.send_and_recv()
            """
            src_msg_sum: torch.Tensor = torch.sum(nodes.mailbox['src_msg'],
                                                  dim=1)
            return {'src_msg_sum': src_msg_sum}

        
        g.send_and_recv(g.edges(),
                        message_func=message_func,
                        reduce_func=reduce_func)

        if self.readout_type == 'set2set':
            batch_mol_hidden_states: torch.Tensor = self.readout_set2set(
                g, g.ndata['src_msg_sum'])
        elif self.readout_type == 'global_sum_pooling':
            batch_mol_hidden_states = dgl.sum_nodes(g, 'src_msg_sum')

        
        return batch_mol_hidden_states

    def forward(
        self, graphs: tuple[DGLGraph]
    ) -> Union[tuple[torch.Tensor, torch.Tensor, torch.Tensor], torch.Tensor]:
        g, qm_g = graphs
        node_feats: torch.Tensor = g.ndata[self.nfeat_name]
        edge_feats: torch.Tensor = g.edata[self.efeat_name]

        qm_node_feats: torch.Tensor = qm_g.ndata[self.nfeat_name]
        qm_edge_feats: torch.Tensor = qm_g.edata[self.efeat_name]

        node_encodings: torch.Tensor = self.mpnn(g, node_feats, edge_feats)

        QM_encodings: torch.Tensor = self.QM_mpnn(qm_g, qm_node_feats, qm_edge_feats)
        qm_g.ndata['node_emb'] = QM_encodings
        molecular_QM_encodings = dgl.sum_nodes(qm_g, 'node_emb')
        molecular_encodings: torch.Tensor = self._readout(
            g, node_encodings, edge_feats)
        if self.readout_type == 'global_sum_pooling':
            molecular_encodings = F.softmax(molecular_encodings, dim=1)

        embeddings: torch.Tensor
        out: torch.Tensor
        embeddings, out = self.ffn(torch.concat((molecular_encodings, molecular_QM_encodings), dim=1))
        
        

        if self.mode == 'classification':
            if self.n_tasks == 1:
                logits: torch.Tensor = out.view(-1, self.n_classes)
            else:
                logits = out.view(-1, self.n_tasks, self.n_classes)
            proba: torch.Tensor = F.sigmoid(
                logits)  
            if self.n_classes == 1:
                proba = proba.squeeze(-1)  
            return proba, logits, embeddings
        else:
            return out


class MPNNPOMModel(TorchModel):

    def __init__(self,
                 n_tasks: int,
                 class_imbalance_ratio: Optional[List] = None,
                 loss_aggr_type: str = 'sum',
                 learning_rate: Union[float, LearningRateSchedule] = 0.001,
                 batch_size: int = 25,
                 node_out_feats: int = 64,
                 edge_hidden_feats: int = 128,
                 edge_out_feats: int = 64,
                 num_step_message_passing: int = 3,
                 mpnn_residual: bool = True,
                 message_aggregator_type: str = 'sum',
                 mode: str = 'regression',
                 number_atom_features: int = 133,
                 number_bond_features: int = 14,
                 n_classes: int = 1,
                 readout_type: str = 'set2set',
                 num_step_set2set: int = 6,
                 num_layer_set2set: int = 3,
                 ffn_hidden_list: List = [300],
                 ffn_embeddings: int = 256,
                 ffn_activation: str = 'relu',
                 ffn_dropout_p: float = 0.0,
                 ffn_dropout_at_input_no_act: bool = True,
                 weight_decay: float = 1e-5,
                 self_loop: bool = False,
                 optimizer_name: str = 'adam',
                 device_name: Optional[str] = None,
                 **kwargs):
        model: nn.Module = MPNNPOM(
            n_tasks=n_tasks,
            node_out_feats=node_out_feats,
            edge_hidden_feats=edge_hidden_feats,
            edge_out_feats=edge_out_feats,
            num_step_message_passing=num_step_message_passing,
            mpnn_residual=mpnn_residual,
            message_aggregator_type=message_aggregator_type,
            mode=mode,
            number_atom_features=number_atom_features,
            number_bond_features=number_bond_features,
            n_classes=n_classes,
            readout_type=readout_type,
            num_step_set2set=num_step_set2set,
            num_layer_set2set=num_layer_set2set,
            ffn_hidden_list=ffn_hidden_list,
            ffn_embeddings=ffn_embeddings,
            ffn_activation=ffn_activation,
            ffn_dropout_p=ffn_dropout_p,
            ffn_dropout_at_input_no_act=ffn_dropout_at_input_no_act)

        if class_imbalance_ratio and (len(class_imbalance_ratio) != n_tasks):
            raise Exception("size of class_imbalance_ratio \
                            should be equal to n_tasks")

        if mode == 'regression':
            loss: Loss = L2Loss()
            output_types: List = ['prediction']
        else:
            loss = CustomMultiLabelLoss(
                class_imbalance_ratio=class_imbalance_ratio,
                loss_aggr_type=loss_aggr_type,
                device=device_name)
            output_types = ['prediction', 'loss', 'embedding']

        optimizer: Optimizer = get_optimizer(optimizer_name)
        optimizer.learning_rate = learning_rate
        if device_name is not None:
            device: Optional[torch.device] = torch.device(device_name)
        else:
            device = None
        super(MPNNPOMModel, self).__init__(model,
                                           loss=loss,
                                           output_types=output_types,
                                           optimizer=optimizer,
                                           learning_rate=learning_rate,
                                           batch_size=batch_size,
                                           device=device,
                                           **kwargs)

        self.weight_decay: float = weight_decay
        self._self_loop: bool = self_loop
        self.regularization_loss: Callable = self._regularization_loss

    def _regularization_loss(self) -> torch.Tensor:
        l1_regularization: torch.Tensor = torch.tensor(0., requires_grad=True)
        l2_regularization: torch.Tensor = torch.tensor(0., requires_grad=True)
        for name, param in self.model.named_parameters():
            if 'bias' not in name:
                l1_regularization = l1_regularization + torch.norm(param, p=1)
                l2_regularization = l2_regularization + torch.norm(param, p=2)
        l1_norm: torch.Tensor = self.weight_decay * l1_regularization
        l2_norm: torch.Tensor = self.weight_decay * l2_regularization
        return l1_norm + l2_norm

    def _prepare_batch(
        self, batch: Tuple[List, List, List]
    ) -> Tuple[DGLGraph, List[torch.Tensor], List[torch.Tensor]]:
        inputs: List
        labels: List
        weights: List

        inputs, labels, weights = batch
        dgl_graphs1: List[DGLGraph] = [
            graphs[0].to_dgl_graph(self_loop=self._self_loop)
            for graphs in inputs[0]
        ]
        g1: DGLGraph = dgl.batch(dgl_graphs1).to(self.device)

        dgl_graphs2: List[DGLGraph] = [
            graphs[1].to_dgl_graph(self_loop=self._self_loop)
            for graphs in inputs[0]
        ]
        g2: DGLGraph = dgl.batch(dgl_graphs2).to(self.device)
        _, labels, weights = super(MPNNPOMModel, self)._prepare_batch(
            ([], labels, weights))
        return (g1, g2), labels, weights


In [ ]:
type(train_dataset)
import os
import deepchem as dc

def convert_to_disk_dataset(train_dataset, output_dir):
    """
    Converts a deepchem.data.datasets.NumpyDataset to a deepchem.data.datasets.DiskDataset.
    
    Parameters:
    train_dataset (deepchem.data.datasets.NumpyDataset): The input NumpyDataset to convert.
    output_dir (str): The directory to save the DiskDataset.
    
    Returns:
    deepchem.data.datasets.DiskDataset: The converted DiskDataset.
    """
    os.makedirs(output_dir, exist_ok=True)
    
    X, y, w, ids = train_dataset.X, train_dataset.y, train_dataset.w, train_dataset.ids
    
    disk_dataset = dc.data.DiskDataset.from_numpy(X, y, w, ids, data_dir=output_dir)
    
    return disk_dataset

output_dir = r'C:\Users\sarab\OneDrive\Desktop\hyperparamter_dmpnn\train_dataset'

train_dataset = convert_to_disk_dataset(train_dataset, output_dir)

In [22]:
train_ratios = get_class_imbalance_ratio(train_dataset)
assert len(train_ratios) == n_tasks

In [ ]:
from deepchem.models.optimizers import ExponentialDecay

learning_rate = ExponentialDecay(initial_rate=0.005390333, decay_rate=0.777099289, decay_steps=764, staircase=True)


In [25]:
import os
import gc
nb_epoch = 40
n_models = 30

In [26]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)

In [ ]:
checkpoint_dir = "./30_ensemble_models/"
os.makedirs(checkpoint_dir, exist_ok=True)
for i in tqdm(range(n_models)):
    model = MPNNPOMModel(n_tasks = n_tasks,
                            batch_size=25,
                            learning_rate=learning_rate,
                            class_imbalance_ratio = train_ratios,
                            loss_aggr_type = 'sum',
                            node_out_feats = 25,
                            edge_hidden_feats = 75,
                            edge_out_feats = 25,
                            num_step_message_passing = 5,
                            mpnn_residual = True,
                            message_aggregator_type = 'max',
                            mode = 'classification',
                            number_atom_features = 133,
                            number_bond_features = 14,
                            n_classes = 1,
                            readout_type = 'set2set',
                            num_step_set2set = 3,
                            num_layer_set2set = 2,
                            ffn_hidden_list= [512, 512],
                            ffn_embeddings = 256,
                            ffn_activation = 'relu',
                            ffn_dropout_p = 0.154493949,
                            ffn_dropout_at_input_no_act = False,
                            weight_decay = 1.39e-6,
                            self_loop = False,
                            optimizer_name = 'adam',
                            log_frequency = 32,
                            model_dir = f'./30_ensemble_models/experiments_{i+1}',
                            device_name='cpu')

    start_time = datetime.now()
    
    
    loss = model.fit(
          train_dataset,
          nb_epoch=nb_epoch,
          max_checkpoints_to_keep=1,
          deterministic=False,
          restore=False)
    end_time = datetime.now()
    
    
    
    
    
    
    
    train_scores = model.evaluate(train_dataset, [metric])['roc_auc_score']
    valid_scores = model.evaluate(valid_dataset, [metric])['roc_auc_score']
    test_scores = model.evaluate(test_dataset, [metric])['roc_auc_score']
    print(f"loss = {loss}; train_scores = {train_scores}; valid_scores = {valid_scores};test_scores = {test_scores}; time_taken = {str(end_time-start_time)}")
    model.save_checkpoint() 
    del model
    gc.collect()
    



 

  0%|          | 0/30 [00:00<?, ?it/s]c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\feat\graph_data.py:193: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:212.)
  g.ndata['x'] = torch.from_numpy(self.node_features).float()


loss = 2.1997361183166504; train_scores = 0.9246231844282585; valid_scores = 0.8669820538385522;test_scores = 0.8603067339965504; time_taken = 0:28:27.818599


  3%|▎         | 1/30 [29:01<14:01:37, 1741.30s/it]

loss = 2.2623543739318848; train_scores = 0.9261822611206043; valid_scores = 0.8652615635470537;test_scores = 0.8601949097178339; time_taken = 0:28:21.638005


  7%|▋         | 2/30 [57:56<13:30:51, 1737.54s/it]

...

 93%|█████████▎| 28/30 [14:10:39<1:00:46, 1823.48s/it]

loss = 2.2283947467803955; train_scores = 0.9261512469804072; valid_scores = 0.8719303279675237;test_scores = 0.864200609342259; time_taken = 0:28:30.666666


 97%|█████████▋| 29/30 [14:39:43<29:59, 1799.66s/it]  

loss = 2.2204537391662598; train_scores = 0.9235849200040607; valid_scores = 0.864358143967663;test_scores = 0.8599051380845463; time_taken = 0:28:25.984980


100%|██████████| 30/30 [15:08:42<00:00, 1817.41s/it]


In [ ]:
list_preds = []
for i in range(n_models):
    model = MPNNPOMModel(n_tasks = n_tasks,
                            batch_size=25,
                            learning_rate=learning_rate,
                            class_imbalance_ratio = train_ratios,
                            loss_aggr_type = 'sum',
                            node_out_feats = 25,
                            edge_hidden_feats = 75,
                            edge_out_feats = 25,
                            num_step_message_passing = 5,
                            mpnn_residual = True,
                            message_aggregator_type = 'max',
                            mode = 'classification',
                            number_atom_features = 133,
                            number_bond_features = 14,
                            n_classes = 1,
                            readout_type = 'set2set',
                            num_step_set2set = 3,
                            num_layer_set2set = 2,
                            ffn_hidden_list= [512, 512],
                            ffn_embeddings = 256,
                            ffn_activation = 'relu',
                            ffn_dropout_p = 0.154493949,
                            ffn_dropout_at_input_no_act = False,
                            weight_decay = 1.39e-6,
                            self_loop = False,
                            optimizer_name = 'adam',
                            log_frequency = 32,
                            model_dir = f'./30_ensemble_models/experiments_{i+1}',
                            device_name='cpu')
    model.restore(f"./30_ensemble_models/experiments_{i+1}/checkpoint2.pt")
    
    
    preds = model.predict(test_dataset)
    list_preds.append(preds)

preds_arr = np.asarray(list_preds)
ensemble_preds = np.mean(preds_arr, axis=0)
print("Mean Test ensemble score: ", roc_auc_score(test_dataset.y, ensemble_preds, average="macro"))

c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\models\torch_models\torch_model.py:1078: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(check

Mean Test ensemble score:  0.8747961566177839


In [ ]:
list_preds = []
for i in range(n_models):
    model = MPNNPOMModel(n_tasks = n_tasks,
                            batch_size=25,
                            learning_rate=learning_rate,
                            class_imbalance_ratio = train_ratios,
                            loss_aggr_type = 'sum',
                            node_out_feats = 25,
                            edge_hidden_feats = 75,
                            edge_out_feats = 25,
                            num_step_message_passing = 5,
                            mpnn_residual = True,
                            message_aggregator_type = 'max',
                            mode = 'classification',
                            number_atom_features = 133,
                            number_bond_features = 14,
                            n_classes = 1,
                            readout_type = 'set2set',
                            num_step_set2set = 3,
                            num_layer_set2set = 2,
                            ffn_hidden_list= [512, 512],
                            ffn_embeddings = 256,
                            ffn_activation = 'relu',
                            ffn_dropout_p = 0.154493949,
                            ffn_dropout_at_input_no_act = False,
                            weight_decay = 1.39e-6,
                            self_loop = False,
                            optimizer_name = 'adam',
                            log_frequency = 32,
                            model_dir = f'./30_ensemble_models/experiments_{i+1}',
                            device_name='cpu')
    model.restore(f"./30_ensemble_models/experiments_{i+1}/checkpoint2.pt")
    
    
    preds = model.predict(test_dataset)
    list_preds.append(preds)

preds_arr = np.asarray(list_preds)
ensemble_preds = np.max(preds_arr, axis=0)
print("Max Test ensemble score: ", roc_auc_score(test_dataset.y, ensemble_preds, average="macro"))

c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\models\torch_models\torch_model.py:1078: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(check

Max Test ensemble score:  0.8690848629860012


In [ ]:
list_preds = []
for i in range(n_models):
    model = MPNNPOMModel(n_tasks = n_tasks,
                            batch_size=25,
                            learning_rate=learning_rate,
                            class_imbalance_ratio = train_ratios,
                            loss_aggr_type = 'sum',
                            node_out_feats = 25,
                            edge_hidden_feats = 75,
                            edge_out_feats = 25,
                            num_step_message_passing = 5,
                            mpnn_residual = True,
                            message_aggregator_type = 'max',
                            mode = 'classification',
                            number_atom_features = 133,
                            number_bond_features = 14,
                            n_classes = 1,
                            readout_type = 'set2set',
                            num_step_set2set = 3,
                            num_layer_set2set = 2,
                            ffn_hidden_list= [512, 512],
                            ffn_embeddings = 256,
                            ffn_activation = 'relu',
                            ffn_dropout_p = 0.154493949,
                            ffn_dropout_at_input_no_act = False,
                            weight_decay = 1.39e-6,
                            self_loop = False,
                            optimizer_name = 'adam',
                            log_frequency = 32,
                            model_dir = f'./30_ensemble_models/experiments_{i+1}',
                            device_name='cpu')
    model.restore(f"./30_ensemble_models/experiments_{i+1}/checkpoint2.pt")
    
    
    preds = model.predict(test_dataset)
    list_preds.append(preds)

preds_arr = np.asarray(list_preds)
ensemble_preds = np.min(preds_arr, axis=0)
print("Min Test ensemble score: ", roc_auc_score(test_dataset.y, ensemble_preds, average="macro"))

c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\models\torch_models\torch_model.py:1078: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(check

Min Test ensemble score:  0.8645668173608708


In [ ]:
list_preds = []
for i in range(n_models):
    model = MPNNPOMModel(n_tasks = n_tasks,
                            batch_size=25,
                            learning_rate=learning_rate,
                            class_imbalance_ratio = train_ratios,
                            loss_aggr_type = 'sum',
                            node_out_feats = 25,
                            edge_hidden_feats = 75,
                            edge_out_feats = 25,
                            num_step_message_passing = 5,
                            mpnn_residual = True,
                            message_aggregator_type = 'max',
                            mode = 'classification',
                            number_atom_features = 133,
                            number_bond_features = 14,
                            n_classes = 1,
                            readout_type = 'set2set',
                            num_step_set2set = 3,
                            num_layer_set2set = 2,
                            ffn_hidden_list= [512, 512],
                            ffn_embeddings = 256,
                            ffn_activation = 'relu',
                            ffn_dropout_p = 0.154493949,
                            ffn_dropout_at_input_no_act = False,
                            weight_decay = 1.39e-6,
                            self_loop = False,
                            optimizer_name = 'adam',
                            log_frequency = 32,
                            model_dir = f'./30_ensemble_models/experiments_{i+1}',
                            device_name='cpu')
    model.restore(f"./30_ensemble_models/experiments_{i+1}/checkpoint2.pt")
    
    
    preds = model.predict(test_dataset)
    list_preds.append(preds)

preds_arr = np.asarray(list_preds)
ensemble_preds = np.median(preds_arr, axis=0)
print("Median Test ensemble score: ", roc_auc_score(test_dataset.y, ensemble_preds, average="macro"))

c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\models\torch_models\torch_model.py:1078: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(check

Median Test ensemble score:  0.8742928158301737


In [ ]:
list_preds = []
for i in range(n_models):
    model = MPNNPOMModel(n_tasks = n_tasks,
                            batch_size=25,
                            learning_rate=learning_rate,
                            class_imbalance_ratio = train_ratios,
                            loss_aggr_type = 'sum',
                            node_out_feats = 25,
                            edge_hidden_feats = 75,
                            edge_out_feats = 25,
                            num_step_message_passing = 5,
                            mpnn_residual = True,
                            message_aggregator_type = 'max',
                            mode = 'classification',
                            number_atom_features = 133,
                            number_bond_features = 14,
                            n_classes = 1,
                            readout_type = 'set2set',
                            num_step_set2set = 3,
                            num_layer_set2set = 2,
                            ffn_hidden_list= [512, 512],
                            ffn_embeddings = 256,
                            ffn_activation = 'relu',
                            ffn_dropout_p = 0.154493949,
                            ffn_dropout_at_input_no_act = False,
                            weight_decay = 1.39e-6,
                            self_loop = False,
                            optimizer_name = 'adam',
                            log_frequency = 32,
                            model_dir = f'./30_ensemble_models/experiments_{i+1}',
                            device_name='cpu')
    model.restore(f"./30_ensemble_models/experiments_{i+1}/checkpoint2.pt")
    
    
    preds = model.predict(valid_dataset)
    list_preds.append(preds)

preds_arr = np.asarray(list_preds)
ensemble_preds = np.mean(preds_arr, axis=0)
print("Mean Valid ensemble score: ", roc_auc_score(valid_dataset.y, ensemble_preds, average="macro"))

c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\models\torch_models\torch_model.py:1078: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(check

Mean Valid ensemble score:  0.8793698319234277


In [ ]:
list_preds = []
for i in range(n_models):
    model = MPNNPOMModel(n_tasks = n_tasks,
                            batch_size=25,
                            learning_rate=learning_rate,
                            class_imbalance_ratio = train_ratios,
                            loss_aggr_type = 'sum',
                            node_out_feats = 25,
                            edge_hidden_feats = 75,
                            edge_out_feats = 25,
                            num_step_message_passing = 5,
                            mpnn_residual = True,
                            message_aggregator_type = 'max',
                            mode = 'classification',
                            number_atom_features = 133,
                            number_bond_features = 14,
                            n_classes = 1,
                            readout_type = 'set2set',
                            num_step_set2set = 3,
                            num_layer_set2set = 2,
                            ffn_hidden_list= [512, 512],
                            ffn_embeddings = 256,
                            ffn_activation = 'relu',
                            ffn_dropout_p = 0.154493949,
                            ffn_dropout_at_input_no_act = False,
                            weight_decay = 1.39e-6,
                            self_loop = False,
                            optimizer_name = 'adam',
                            log_frequency = 32,
                            model_dir = f'./30_ensemble_models/experiments_{i+1}',
                            device_name='cpu')
    model.restore(f"./30_ensemble_models/experiments_{i+1}/checkpoint2.pt")
    
    
    preds = model.predict(valid_dataset)
    list_preds.append(preds)

preds_arr = np.asarray(list_preds)
ensemble_preds = np.max(preds_arr, axis=0)
print("Max Valid ensemble score: ", roc_auc_score(valid_dataset.y, ensemble_preds, average="macro"))

c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\models\torch_models\torch_model.py:1078: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(check

Max Valid ensemble score:  0.8707364997960599


In [ ]:
list_preds = []
for i in range(n_models):
    model = MPNNPOMModel(n_tasks = n_tasks,
                            batch_size=25,
                            learning_rate=learning_rate,
                            class_imbalance_ratio = train_ratios,
                            loss_aggr_type = 'sum',
                            node_out_feats = 25,
                            edge_hidden_feats = 75,
                            edge_out_feats = 25,
                            num_step_message_passing = 5,
                            mpnn_residual = True,
                            message_aggregator_type = 'max',
                            mode = 'classification',
                            number_atom_features = 133,
                            number_bond_features = 14,
                            n_classes = 1,
                            readout_type = 'set2set',
                            num_step_set2set = 3,
                            num_layer_set2set = 2,
                            ffn_hidden_list= [512, 512],
                            ffn_embeddings = 256,
                            ffn_activation = 'relu',
                            ffn_dropout_p = 0.154493949,
                            ffn_dropout_at_input_no_act = False,
                            weight_decay = 1.39e-6,
                            self_loop = False,
                            optimizer_name = 'adam',
                            log_frequency = 32,
                            model_dir = f'./30_ensemble_models/experiments_{i+1}',
                            device_name='cpu')
    model.restore(f"./30_ensemble_models/experiments_{i+1}/checkpoint2.pt")
    
    
    preds = model.predict(valid_dataset)
    list_preds.append(preds)

preds_arr = np.asarray(list_preds)
ensemble_preds = np.min(preds_arr, axis=0)
print("Min Valid ensemble score: ", roc_auc_score(valid_dataset.y, ensemble_preds, average="macro"))

c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\models\torch_models\torch_model.py:1078: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(check

Min Valid ensemble score:  0.8737619850412229


In [ ]:
list_preds = []
for i in range(n_models):
    model = MPNNPOMModel(n_tasks = n_tasks,
                            batch_size=25,
                            learning_rate=learning_rate,
                            class_imbalance_ratio = train_ratios,
                            loss_aggr_type = 'sum',
                            node_out_feats = 25,
                            edge_hidden_feats = 75,
                            edge_out_feats = 25,
                            num_step_message_passing = 5,
                            mpnn_residual = True,
                            message_aggregator_type = 'max',
                            mode = 'classification',
                            number_atom_features = 133,
                            number_bond_features = 14,
                            n_classes = 1,
                            readout_type = 'set2set',
                            num_step_set2set = 3,
                            num_layer_set2set = 2,
                            ffn_hidden_list= [512, 512],
                            ffn_embeddings = 256,
                            ffn_activation = 'relu',
                            ffn_dropout_p = 0.154493949,
                            ffn_dropout_at_input_no_act = False,
                            weight_decay = 1.39e-6,
                            self_loop = False,
                            optimizer_name = 'adam',
                            log_frequency = 32,
                            model_dir = f'./30_ensemble_models/experiments_{i+1}',
                            device_name='cpu')
    model.restore(f"./30_ensemble_models/experiments_{i+1}/checkpoint2.pt")
    
    
    preds = model.predict(valid_dataset)
    list_preds.append(preds)

preds_arr = np.asarray(list_preds)
ensemble_preds = np.median(preds_arr, axis=0)
print("Median Valid ensemble score: ", roc_auc_score(valid_dataset.y, ensemble_preds, average="macro"))

c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\models\torch_models\torch_model.py:1078: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(check

Median Valid ensemble score:  0.8785904945076761
